In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import pandas as pd
from glob import glob 
from fastcore.xtras import load_pickle

from gpt3forchem.output import get_regression_metrics
from gpt3forchem.api_wrappers import extract_prediction

## Classification

In [9]:
all_res = glob('results/20221129_esol/*')

In [13]:
metrics = []
baselines = []

for res in all_res:
    res = load_pickle(res)
    metrics.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': res['cm'].ACC_Macro,
            'f1_macro': res['cm'].F1_Macro,
            'f1_micro': res['cm'].F1_Micro
        })
    baselines.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': res['baseline']['cm'].ACC_Macro,
            'f1_macro': res['baseline']['cm'].F1_Macro,
            'f1_micro': res['baseline']['cm'].F1_Micro  
        })

In [30]:
res = load_pickle(all_res[-1])

In [31]:
[extract_prediction(res['completions'], i) for i in range(len(res['completions']['choices']))]

['large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'large',
 'medium',
 'large',
 'large',
 'large',
 'large',

In [15]:
metrics = pd.DataFrame(metrics)
baselines = pd.DataFrame(baselines)

In [17]:
metrics.groupby(['representation', 'train_size']).agg(['mean', 'std'])

accuracy           f1_macro      f1_micro     
                               mean       std     mean  std     mean  std
representation train_size                                                
inchi          10          0.714286       NaN      0.0  NaN      0.0  NaN
               50          0.714286       NaN      0.0  NaN      0.0  NaN
               500         0.800000       NaN      0.0  NaN      0.0  NaN
selfies        10          0.750000       NaN      0.0  NaN      0.0  NaN
               50          0.714286       NaN      0.0  NaN      0.0  NaN
               500         0.800000       NaN      0.0  NaN      0.0  NaN
smiles         10          0.690476  0.033672      0.0  0.0      0.0  0.0
               50          0.750000       NaN      0.0  NaN      0.0  NaN
               500         0.800000       NaN      0.0  NaN      0.0  NaN

In [18]:
baselines.groupby(['train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_65015/4006146815.py:1: FutureWarning: ['representation'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  baselines.groupby(['train_size']).agg(['mean', 'std'])


accuracy            f1_macro            f1_micro          
                mean       std      mean       std      mean       std
train_size                                                            
10          0.744000  0.014697  0.105725  0.007744  0.360000  0.036742
50          0.840000  0.020785  0.418630  0.113997  0.600000  0.051962
500         0.907333  0.013013  0.717476  0.044243  0.768333  0.032532